<a href="https://colab.research.google.com/github/oreushka/SIA_lab/blob/main/Month2_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Задание

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [ ]:
 import random 
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры Rock и Country


Прочитаем данные из датасета

In [ ]:
import pandas as pd
data = pd.read_csv("dataset.csv")
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


Полученные жанры были Rock и Country. Их и выберем из датасета

In [ ]:
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Rock') | (columns.genre == 'Country')]

Приведем тексты к нижнему регистру и сделаем с ними отдельную колонку

In [ ]:
lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

Проведем токенизацию и добавим токенизированные данные в колонку

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

Удалим стоп-слова или шумы. Опять же сделаем колонку с обновленными данными

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

Проведем лемматизацию

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

Разделим данные на обучающую и тестовые выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки 

> 



y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre


> 


x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр


> 



y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

Проведем векторизацию(представим текст в численном виде)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Проведем классификацию, используя Наивный Байесовский Классификатор

In [ ]:
#импортируем байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

Посмотрим предсказания тестовой выборки

In [ ]:
clf.predict(vectorized_x_test)

array(['Rock', 'Country', 'Country', 'Country', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Country', 'Country', 'Rock',
       'Rock', 'Country', 'Rock', 'Rock', 'Rock', 'Rock', 'Country',
       'Country', 'Rock', 'Rock', 'Rock', 'Country', 'Country', 'Rock',
       'Country', 'Rock', 'Country', 'Country', 'Country', 'Rock', 'Rock',
       'Country', 'Country', 'Country', 'Rock', 'Country', 'Rock', 'Rock',
       'Country', 'Rock', 'Rock', 'Country', 'Country', 'Rock', 'Rock',
       'Rock', 'Country', 'Rock', 'Rock', 'Country', 'Country'],
      dtype='<U7')

Получим оценки классификации

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     Country       0.67      0.62      0.64        26
        Rock       0.68      0.72      0.70        29

    accuracy                           0.67        55
   macro avg       0.67      0.67      0.67        55
weighted avg       0.67      0.67      0.67        55



2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

Импортируем данные

In [ ]:
data = pd.read_csv("TheBestDataset.csv")
data.head()

,genre,lyrics
0,Country,I hear songs on the radio They might be fast o...
1,Rock,Empty spaces what are we living for? Abandone...


In [ ]:
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Rock') | (columns.genre == 'Country')]

Выполним обработку: приведем к нижнему регистру, проведем токенизацию, удалим стоп-слова, проведем лемматизацию

In [ ]:
lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

In [ ]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Создадим тестовую выборку

In [ ]:
x_test = columns['lemmatized']

Векторизуем ее

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)

И выполним предсказание жанра

In [ ]:
clf.predict(vectorized_x_test)

array(['Rock', 'Rock'], dtype='<U7')

Обе песни получились жанра Rock. На то он и наивный, этот Байесовский Классификатор :) Ну или обучающая выборка была маленькая
Получим оценки классификации, предварительно создав переменную с жанрами и векторизовав ее 

In [ ]:
y_test = columns['genre']
vectorized_y_test = vectorizer.transform(y_test)

In [ ]:
print(classification_report(y_test, clf.predict(vectorized_x_test)))

              precision    recall  f1-score   support

     Country       0.00      0.00      0.00         1
        Rock       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни 
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

Импортируем датасет и посмотри, что там вообще есть

In [ ]:
data = pd.read_csv("dataset-lyrics-musics-mini.csv")
data.head()

,cantorId,cantorNome,musicaNome,letra
0,0,david-bowie,Heroes,"I, I will be king. And you, you will be queen...."
1,0,david-bowie,Starman,"Didn't know what time it was,. The lights were..."
2,0,david-bowie,Space Oddity,Ground control to Major Tom. Ground control to...
3,0,david-bowie,Life On Mars?,It's a god-awful small affair. To the girl wit...
4,0,david-bowie,Modern Love,I know when to go out. And when to stay in. Ge...


Узнаем имена певцов в датасете

In [ ]:
data['cantorNome'].unique()

array(['david-bowie', 'paul-mccartney', 'laura-pausini', 'evanescence',
       'ac-dc'], dtype=object)

Создадим таблицу поменьше с нужными нам данными

In [ ]:
#поскольку нам нужны только имена и тексты, их и оставим
columns = data[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'david-bowie') | (columns.cantorNome == 'paul-mccartney')]

Выполним обработку текста песен: приведем к нижнему регистру, проведем токенизацию, удалим стоп-слова, проведем лемматизацию

In [ ]:
lowered = columns['letra'].str.lower()
columns['lowered'] = lowered

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

In [ ]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделим данные на обучающую и тестовые выборки, проведем их векторизацию

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Проведем классификацию с Наивным Байесовским классификатором, векторизуем тестовую выборку, посмотрим предсказания на тестовой выборке и получим оценки классификаиции

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
clf.predict(vectorized_x_test)

array(['david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'paul-mccartney', 'david-bowie', 'david-bowie',
       'david-bowie', 'david-bowie', 'david-bowie', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'david-bowie',
       'david-bowie', 'paul-mccartney', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'paul-mccartney', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'david-bowie', 'david-bowie', 'paul-mccartney', 'paul-mccartney',
       'david-bowie', 'paul-mccartney', 'david-bowie', 'paul-mccartney',
       'david-bowie', 'david-bowie', 'david-bowie', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'pau

In [ ]:
print(classification_report(y_test, clf.predict(vectorized_x_test)))

                precision    recall  f1-score   support

   david-bowie       0.75      0.73      0.74       146
paul-mccartney       0.73      0.75      0.74       139

      accuracy                           0.74       285
     macro avg       0.74      0.74      0.74       285
  weighted avg       0.74      0.74      0.74       285

